# Building Machine Learning Classifiers: Explore Random Forest model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,0.047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,0.041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,0.032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,0.071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,0.044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Build our own Grid-search

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [3]:
# Convert column names to strings
X_features.columns = X_features.columns.astype(str)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [5]:
# Define a function for estimater and depth
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [6]:
# using for loop for iterations of estimaters and depth
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.221 / Accuracy: 0.896
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.57 / Accuracy: 0.943
Est: 10 / Depth: 30 ---- Precision: 0.991 / Recall: 0.752 / Accuracy: 0.966
Est: 10 / Depth: None ---- Precision: 0.992 / Recall: 0.846 / Accuracy: 0.978
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.215 / Accuracy: 0.895
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.57 / Accuracy: 0.943
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.745 / Accuracy: 0.966
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.866 / Accuracy: 0.982
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.221 / Accuracy: 0.896
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.597 / Accuracy: 0.946
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.725 / Accuracy: 0.963
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.852 / Accuracy: 0.98
